In [ ]:
import os
import pandas as pd
import re
import math
import copy
import numpy as np
import sys
# Add the root directory /workspaces/llm_etl to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))
# Now import your module
from spider2_utils import load_csv_database
# Load environment variables


-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("modern_data", rows_limit=-1)

pizza_clean_customer_orders = _database["pizza_clean_customer_orders"]
pizza_recipes = _database["pizza_recipes"]
pizza_toppings = _database["pizza_toppings"]

### Question
Based on our customer pizza order information, summarize the total quantity of each ingredient used in the pizzas we delivered. Output the name and quantity for each ingredient.

### Step 1: Add row number to `pizza_clean_customer_orders`

In [ ]:
pizza_clean_customer_orders = pizza_clean_customer_orders.copy()
pizza_clean_customer_orders['original_row_number'] = range(1, len(pizza_clean_customer_orders) + 1)

### Step 2: Recursively split toppings in `pizza_recipes`

In [ ]:
topping_rows = []
for _, row in pizza_recipes.iterrows():
    toppings = row['toppings'].split(',') if pd.notnull(row['toppings']) else []
    for topping in toppings:
        topping_rows.append({'pizza_id': row['pizza_id'], 'topping_id': topping.strip()})
split_regular_toppings = pd.DataFrame(topping_rows)

### Step 3: Join regular toppings with customer orders to get base toppings

In [ ]:
cte_base_toppings = pizza_clean_customer_orders.merge(split_regular_toppings, on='pizza_id', how='left')

### Step 4: Recursively split `exclusions`

In [ ]:
exclusion_rows = []
for _, row in pizza_clean_customer_orders.iterrows():
    if pd.notnull(row['exclusions']):
        toppings = row['exclusions'].split(',')
        for topping in toppings:
            exclusion_rows.append({**row.to_dict(), 'topping_id': topping.strip()})
split_exclusions = pd.DataFrame(exclusion_rows)

### Step 5: Recursively split `extras`

In [ ]:
extra_rows = []
for _, row in pizza_clean_customer_orders.iterrows():
    if pd.notnull(row['extras']):
        toppings = row['extras'].split(',')
        for topping in toppings:
            extra_rows.append({**row.to_dict(), 'topping_id': topping.strip()})
split_extras = pd.DataFrame(extra_rows)

### Step 6: Filter base toppings by removing exclusions

In [ ]:
cte_base_filtered = cte_base_toppings.merge(
    split_exclusions[['order_id', 'topping_id']],
    on=['order_id', 'topping_id'],
    how='left',
    indicator=True
)
cte_base_filtered = cte_base_filtered[cte_base_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])

### Step 7: Combine filtered base toppings with extras

In [ ]:
cte_combined_orders = pd.concat([
    cte_base_filtered[['order_id', 'customer_id', 'pizza_id', 'order_time', 'original_row_number', 'topping_id']],
    split_extras[['order_id', 'customer_id', 'pizza_id', 'order_time', 'original_row_number', 'topping_id']]
], ignore_index=True)

### Step 8: Join with `pizza_toppings` to get topping names and count frequency

In [ ]:
# Ensure both `topping_id` columns are of the same type (string is safer for IDs like '1', '2', '12')
cte_combined_orders['topping_id'] = cte_combined_orders['topping_id'].astype(str)
pizza_toppings['topping_id'] = pizza_toppings['topping_id'].astype(str)

# Perform the merge
combined_orders = cte_combined_orders.merge(pizza_toppings, on='topping_id', how='left')

# Group by topping name and count
topping_counts = combined_orders.groupby('topping_name').size().reset_index(name='topping_count')
topping_counts = topping_counts.sort_values(by='topping_count', ascending=False)
topping_counts